In [81]:
import typing  # noqa: N999
from datetime import datetime, date
from openpyxl import load_workbook
import importlib
import math

from pathlib import Path

import pandas as pd
from crossasset.helpers.matrix import Matrix
from crossasset.market_data.dividend import Dividend
from crossasset.market_data.eq_vol import EqVol

from equity_vol import EquityVol

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
asofdt = datetime(year=2024, month=10, day=1).date()
print(asofdt.strftime('%Y%m%d'))

my_date_str = '20241001'
my_date = datetime.strptime(my_date_str, '%Y%m%d').date()
print(my_date)

20241001
2024-10-01


In [3]:
if isinstance(asofdt, date):
    print("It's a date!")
else:
    print(type(asofdt))

It's a date!


In [56]:
idx_name = 'SPX Index'
idx_spot = 5708.75

spx_vol = EquityVol(my_date_str, idx_name, idx_spot)

All_Dts A2:A14
Expiry_Dts A3:A14
Strikes B2:N2
Full_IV_Matrix A2:N14
IV_Data B13:N14
YC_DFs S3:S14
Dvd_Rts R3:R14


In [35]:
vol_file = spx_vol.vol_file
vol_sheet = spx_vol.sheet_name
all_dts_rng = spx_vol.sheet_rng('All_Dts')
vol_file, vol_sheet, all_dts_rng

(WindowsPath('//rgare.net/stlfinmkts/MarketRiskMgmt/Market Hedge VA/Markit Vol Surface/Markit_Vol_Surface_20241001.xlsx'),
 'S&P500',
 'A2:A14')

In [68]:
test_excel_data = spx_vol.read_all_excel_data()

In [59]:
for key, val in test_excel_data.items():
    print(key, val)

All_Dts [datetime.datetime(2024, 10, 1, 0, 0), datetime.datetime(2024, 10, 18, 0, 0), datetime.datetime(2024, 11, 15, 0, 0), datetime.datetime(2024, 12, 20, 0, 0), datetime.datetime(2025, 3, 21, 0, 0), datetime.datetime(2025, 6, 20, 0, 0), datetime.datetime(2025, 9, 19, 0, 0), datetime.datetime(2025, 12, 19, 0, 0), datetime.datetime(2026, 6, 18, 0, 0), datetime.datetime(2026, 12, 18, 0, 0), datetime.datetime(2027, 6, 17, 0, 0), datetime.datetime(2027, 12, 17, 0, 0), datetime.datetime(2028, 12, 15, 0, 0)]
Expiry_Dts [datetime.datetime(2024, 10, 18, 0, 0), datetime.datetime(2024, 11, 15, 0, 0), datetime.datetime(2024, 12, 20, 0, 0), datetime.datetime(2025, 3, 21, 0, 0), datetime.datetime(2025, 6, 20, 0, 0), datetime.datetime(2025, 9, 19, 0, 0), datetime.datetime(2025, 12, 19, 0, 0), datetime.datetime(2026, 6, 18, 0, 0), datetime.datetime(2026, 12, 18, 0, 0), datetime.datetime(2027, 6, 17, 0, 0), datetime.datetime(2027, 12, 17, 0, 0), datetime.datetime(2028, 12, 15, 0, 0)]
Strikes [0.5, 0

In [69]:
disc_fctrs = test_excel_data['YC_DFs']
disc_fctrs.insert(0, 1.0000)
disc_fctrs


[1.0,
 0.997713,
 0.994015,
 0.989732,
 0.979805,
 0.971239,
 0.963513,
 0.95625,
 0.942379,
 0.928593,
 0.914962,
 0.901254,
 0.874195]

In [ ]:
# get the dividend data
all_dates = test_excel_data['All_Dts']
div_dates = test_excel_data["Expiry_Dts"]
div_spot_rts = test_excel_data['Dvd_Rts']

start_dt = datetime.combine(spx_vol.now_date, datetime.min.time())
div_fwd_rts = []
div_dates, div_spot_rts

exp_times = [(dt - start_dt).days/365.25 for dt in div_dates]
cont_spot_rts = [math.log((1+div_spot_rts[t])**exp_times[t])/exp_times[t] for t in range(len(exp_times))]
# all_times = [(dt - start_dt).days/365.25 for dt in div_dates]
dvd_accum_fctrs = [math.exp(cont_spot_rts[t]*exp_times[t]) for t in range(len(exp_times))]
dvd_accum_fctrs.insert(0, 1.0)
chg_in_t = [exp_times[t] if t == 0 else exp_times[t]-exp_times[t-1] for t in range(len(exp_times))]
div_fwd_rts = [math.log(dvd_accum_fctrs[t+1]/dvd_accum_fctrs[t])/chg_in_t[t] for t in range(len(exp_times))]
div_fwd_rts



# exp_times


[0.0031829291024914096,
 0.005265987044588998,
 0.0071907502892823525,
 0.004979180837585964,
 0.005178796963925092,
 0.00627239223402777,
 0.005995523214333116,
 0.0045954185560713665,
 0.005956235649136494,
 0.003317522229543268,
 0.0033124727436448296,
 0.0034905813888080556]

In [ ]:

dvd_accum_fctrs

[1.0,
 1.0001481555180718,
 1.0005519864623176,
 1.0012416564385715,
 1.0024845026247597,
 1.0037788114975732,
 1.0053486746186626,
 1.0068515375433738,
 1.0091470172458843,
 1.0121630451601502,
 1.0138284112142246,
 1.0155123956438232,
 1.0190511445798198]

In [76]:
spx_vol.now_date

datetime.date(2024, 10, 1)

In [86]:
dvd_accum_fctrs = [math.exp(cont_spot_rts[t]*exp_times[t]) for t in range(len(exp_times))].insert(0, 1.0)
dvd_accum_fctrs


In [74]:
test_excel_data["Expiry_Dts"]

[datetime.datetime(2024, 10, 18, 0, 0),
 datetime.datetime(2024, 11, 15, 0, 0),
 datetime.datetime(2024, 12, 20, 0, 0),
 datetime.datetime(2025, 3, 21, 0, 0),
 datetime.datetime(2025, 6, 20, 0, 0),
 datetime.datetime(2025, 9, 19, 0, 0),
 datetime.datetime(2025, 12, 19, 0, 0),
 datetime.datetime(2026, 6, 18, 0, 0),
 datetime.datetime(2026, 12, 18, 0, 0),
 datetime.datetime(2027, 6, 17, 0, 0),
 datetime.datetime(2027, 12, 17, 0, 0),
 datetime.datetime(2028, 12, 15, 0, 0)]

In [55]:
# test = pd.read_excel(vol_file, vol_sheet, usecols=all_dts_rng)
data = None

wb = load_workbook(filename=vol_file, read_only=True)
ws = wb[vol_sheet]
# data = ws[all_dts_rng]


# Extract data into a list of lists
data = []
for row in ws[all_dts_rng]:
    row_data = []
    for cell in row:
        row_data.append(cell.value)
    data.append(row_data)

# Extract data into 1d
data = []
for row in ws[all_dts_rng]:    
    for cell in row:
        data.append(cell.value)
    


data
# for index, item in enumerate(data):
#     print(f"Index: {index}, Value: {item}")
#     for cell in item:
#         print(cell.value)

[datetime.datetime(2024, 10, 1, 0, 0),
 datetime.datetime(2024, 10, 18, 0, 0),
 datetime.datetime(2024, 11, 15, 0, 0),
 datetime.datetime(2024, 12, 20, 0, 0),
 datetime.datetime(2025, 3, 21, 0, 0),
 datetime.datetime(2025, 6, 20, 0, 0),
 datetime.datetime(2025, 9, 19, 0, 0),
 datetime.datetime(2025, 12, 19, 0, 0),
 datetime.datetime(2026, 6, 18, 0, 0),
 datetime.datetime(2026, 12, 18, 0, 0),
 datetime.datetime(2027, 6, 17, 0, 0),
 datetime.datetime(2027, 12, 17, 0, 0),
 datetime.datetime(2028, 12, 15, 0, 0)]

datetime.date(2024, 10, 1)